# TV2 scraper

First, getting updates, libraries, chromedriver, etc.

In [ ]:
#
!apt-get update
!apt-get install -y wget unzip libvulkan1

!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y

!apt-get install -y xvfb

!pip install selenium chromedriver-autoinstaller

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [59.5 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,107 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu 

Scaping:

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests


import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)


# search terms to search for on outlets' website
keywords = ['facial recognition', 'face recognition', 'ansigtsaftryk', 'ansigtsscan',
                'masseovervågning ansigt', 'masseovervågning kamera', 'ansigtskamera', 'overvågningssamfund ansigt',
                'overvågningssamfund kamera', 'sikkerhedspakke overvågning', 'tryghedsskabende kamera']


# As tv2 restricted its results to only the first 100 articles, I needed to get creative
# for example, i sort using both relevance and date to get as many articles as possible
# here they are with two different URLs
base_url_relevance = 'https://search.tv2.dk/?query={}&sort=relevance&page={}'
base_url_date = 'https://search.tv2.dk/?query={}&sort=date&page={}'

# function to scrape_article
def scrape_article(article_url, keyword):
    try:
        article_response = requests.get(article_url)
        article_soup = BeautifulSoup(article_response.content, 'html.parser')

        # header
        header = article_soup.find('h1').get_text(strip=True)

        # subheader
        subheader = article_soup.find('p', class_='tc_standfirst').get_text(strip=True) if article_soup.find('p', class_='tc_standfirst') else 'No subheader'

        # category
        category = article_soup.find('div', class_='tc_page__header__label').get_text(strip=True) if article_soup.find('div', class_='tc_page__header__label') else 'No category'

        # content
        content_div = article_soup.find('div', class_='tc_page__body')
        content = content_div.get_text(strip=True) if content_div else 'Content not found'

        # date
        date = article_soup.find('time')['datetime'] if article_soup.find('time') else 'No date'


        return {'URL': article_url,
                'Keyword': keyword,
                'Header': header,
                'Subheader': subheader,
                'Category': category,
                'Content': content,
                'Date': date}

    except Exception as e:
        print(f"Error scraping {article_url}: {e}")
        return None


def scrape_page(page_url, keyword):
    try:
        response = requests.get(page_url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # get links on page
        articles = soup.find_all('li', class_='tc_searchresults__item__item')
        article_urls = [article.find('a')['href'] for article in articles]


        page_articles = []

        # scrape each utl by looping through
        for article_url in article_urls:
            data = scrape_article(article_url, keyword)
            if data:
                page_articles.append(data)

        return page_articles

    except Exception as e:
        print(f"Error scraping page {page_url}: {e}")
        return []


# To avoid duplicates
unique_articles = []

# Function to scrape for either sorting type aka relevance and date
def scrape_for_sort_type(base_url, sort_type):

    for keyword in keywords:
        print(f"Scraping for keyword: {keyword} with sort type: {sort_type}")

        current_page = 1  # Startting from p 1

        while True:
            print(f"Scraping page {current_page} for keyword {keyword} with sort type {sort_type}...")

            page_url = base_url.format(keyword, current_page)

            articles_on_page = scrape_page(page_url, keyword)

            if not articles_on_page:
                print(f"No more articles found for keyword {keyword} on page {current_page} with sort type {sort_type}.")
                break

            for article in articles_on_page:
                if article['URL'] not in [art['URL'] for art in unique_articles]:
                    unique_articles.append(article)

            soup = BeautifulSoup(requests.get(page_url).content, 'html.parser')
            next_button = soup.find('a', class_='tc_button tc_pagination__next tc_button--dark tc_button--secondary tc_button--default tc_button--has-icon-only')

            if next_button is None:
                print(f"No more pages for keyword {keyword} with sort type {sort_type}.")
                break  # Stop loop when no next page button

            time.sleep(2)

            current_page += 1


scrape_for_sort_type(base_url_relevance, 'relevance')

scrape_for_sort_type(base_url_date, 'date')

df = pd.DataFrame(unique_articles)

df.to_csv('unique_tv2_articles_all.csv', index=False)
from google.colab import files
files.download('unique_tv2_articles_all.csv')

df.head()


Scraping for keyword: facial recognition with sort type: relevance
Scraping page 1 for keyword facial recognition with sort type relevance...
No more pages for keyword facial recognition with sort type relevance.
Scraping for keyword: face recognition with sort type: relevance
Scraping page 1 for keyword face recognition with sort type relevance...
Scraping page 2 for keyword face recognition with sort type relevance...
No more pages for keyword face recognition with sort type relevance.
Scraping for keyword: ansigtsaftryk with sort type: relevance
Scraping page 1 for keyword ansigtsaftryk with sort type relevance...
No more articles found for keyword ansigtsaftryk on page 1 with sort type relevance.
Scraping for keyword: ansigtsscan with sort type: relevance
Scraping page 1 for keyword ansigtsscan with sort type relevance...
Scraping page 2 for keyword ansigtsscan with sort type relevance...
No more pages for keyword ansigtsscan with sort type relevance.
Scraping for keyword: masseove

Error scraping https://sr.tv2.dk/media/3eupsr5b/dag-ins.pdf: 'NoneType' object has no attribute 'get_text'
Scraping page 6 for keyword overvågningssamfund ansigt with sort type relevance...


Error scraping https://sr.tv2.dk/media/ec2du2f3/tv-2-faar-ikke-kritik-for-trailer-til-kritisk-dokumentar-om-daginstitutioner.pdf: 'NoneType' object has no attribute 'get_text'
Scraping page 7 for keyword overvågningssamfund ansigt with sort type relevance...
Scraping page 8 for keyword overvågningssamfund ansigt with sort type relevance...
Scraping page 9 for keyword overvågningssamfund ansigt with sort type relevance...
Scraping page 10 for keyword overvågningssamfund ansigt with sort type relevance...
Scraping page 11 for keyword overvågningssamfund ansigt with sort type relevance...
No more articles found for keyword overvågningssamfund ansigt on page 11 with sort type relevance.
Scraping for keyword: overvågningssamfund kamera with sort type: relevance
Scraping page 1 for keyword overvågningssamfund kamera with sort type relevance...
Scraping page 2 for keyword overvågningssamfund kamera with sort type relevance...
Scraping page 3 for keyword overvågningssamfund kamera with sort ty

Error scraping https://sr.tv2.dk/media/3eupsr5b/dag-ins.pdf: 'NoneType' object has no attribute 'get_text'
Scraping page 10 for keyword overvågningssamfund ansigt with sort type date...


Error scraping https://sr.tv2.dk/media/ec2du2f3/tv-2-faar-ikke-kritik-for-trailer-til-kritisk-dokumentar-om-daginstitutioner.pdf: 'NoneType' object has no attribute 'get_text'
Scraping page 11 for keyword overvågningssamfund ansigt with sort type date...
No more articles found for keyword overvågningssamfund ansigt on page 11 with sort type date.
Scraping for keyword: overvågningssamfund kamera with sort type: date
Scraping page 1 for keyword overvågningssamfund kamera with sort type date...
Error scraping https://tv2.dk/reel/2024-06-26-overvaagningskamera-fanger-katastrofalt-oejeblik-6355704178112: 'NoneType' object has no attribute 'get_text'
Scraping page 2 for keyword overvågningssamfund kamera with sort type date...
Scraping page 3 for keyword overvågningssamfund kamera with sort type date...
Scraping page 4 for keyword overvågningssamfund kamera with sort type date...
Scraping page 5 for keyword overvågningssamfund kamera with sort type date...
Scraping page 6 for keyword overvåg

Error scraping https://sr.tv2.dk/media/3eupsr5b/dag-ins.pdf: 'NoneType' object has no attribute 'get_text'


Error scraping https://sr.tv2.dk/media/ec2du2f3/tv-2-faar-ikke-kritik-for-trailer-til-kritisk-dokumentar-om-daginstitutioner.pdf: 'NoneType' object has no attribute 'get_text'
Scraping page 9 for keyword overvågningssamfund kamera with sort type date...
Error scraping https://nyheder.tv2.dk/samfund/2020-07-28-disse-skjulte-optagelser-paa-plejehjem-afsloerer-umenneskelig-behandling-af-90: 'NoneType' object has no attribute 'get_text'
Scraping page 10 for keyword overvågningssamfund kamera with sort type date...
Scraping page 11 for keyword overvågningssamfund kamera with sort type date...
No more articles found for keyword overvågningssamfund kamera on page 11 with sort type date.
Scraping for keyword: sikkerhedspakke overvågning with sort type: date
Scraping page 1 for keyword sikkerhedspakke overvågning with sort type date...
Scraping page 2 for keyword sikkerhedspakke overvågning with sort type date...
Scraping page 3 for keyword sikkerhedspakke overvågning with sort type date...
Scr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,URL,Keyword,Header,Subheader,Category,Content,Date
0,https://nyheder.tv2.dk/krimi/2024-08-08-rigspo...,facial recognition,Rigspolitichef forsøger at berolige befolkning...,Ansigtsgenkendelse vil spare politiet ressourc...,Krimi,Det vil gøre politiets arbejde betydeligt nemm...,2024-08-08T14:16:16.655Z
1,https://nyheder.tv2.dk/tech/2019-01-20-der-kan...,facial recognition,Der kan være en bagtanke med '10yearchallenge'...,Man kan bruge ansigtsgenkendelse til mere og m...,Tech,"Mærkelige frisurer, orange kulør fra solariet ...",2019-01-20T15:51:09.000Z
2,https://echo.tv2.dk/samfund/2020-01-07-24-aari...,facial recognition,24-årig kineser om omdiskuteret teknologi: - D...,"I Kina bruges ansigtsgenkendelse flittigt, men...",Samfund,Den 25-årige kinesiske studerende Hong Yen får...,2020-01-07T09:45:31.000Z
3,https://nyheder.tv2.dk/2015-02-02-forbrugerraa...,facial recognition,Forbrugerrådet Tænk til lufthavn: Begrund pass...,No subheader,Nyheder,Københavns Lufthavn har netop fået sat reklame...,2015-02-02T15:43:41.000Z
4,https://nyheder.tv2.dk/tech/2009-08-04-test-et...,facial recognition,Test: Et helt unikt feriekamera,No subheader,Tech,Jeg har ikke tal på hvor mange ferier og andre...,2009-08-04T12:50:00.000Z


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving unique_tv2_articles_2 (1).csv to unique_tv2_articles_2 (1).csv


In [ ]:
import pandas as pd
df = pd.read_csv('unique_tv2_articles_2 (1).csv')

In [ ]:
df

,URL,Keyword,Header,Subheader,Category,Content,Date
0,https://nyheder.tv2.dk/krimi/2024-08-08-rigspo...,ansigtsgenkendelse,Rigspolitichef forsøger at berolige befolkning...,Ansigtsgenkendelse vil spare politiet ressourc...,Krimi,Det vil gøre politiets arbejde betydeligt nemm...,2024-08-08T14:16:16.655Z
1,https://sport.tv2.dk/fodbold/2024-09-06-fc-koe...,ansigtsgenkendelse,FC København vil benytte sig af ansigtsgenkend...,FC København afventer godkendelse til at benyt...,3F Superliga,"Flere har haft på fornemmelsen, at der kunne v...",2024-09-10T05:06:00.865Z
2,https://nyheder.tv2.dk/krimi/2024-08-31-politi...,ansigtsgenkendelse,Politiet tager hul på første projekt med ansig...,Rigspolitiet har hentet tilbud på software til...,Krimi,"Politiet bekræfter, at man er i gang med det f...",2024-08-31T15:01:19.044Z
3,https://nyheder.tv2.dk/udland/2023-09-21-lufth...,ansigtsgenkendelse,Lufthavn i Singapore erstatter pas med ansigts...,Fra 2024 vil man i Changi Lufthavn i Singapore...,Udland,Fra næste år bliver det muligt at rejse gennem...,2023-09-21T04:08:13.728Z
4,https://nyheder.tv2.dk/video/2024-08-12-ansigt...,ansigtsgenkendelse,"Ansigtsgenkendelse er på alle måder en ""god id...",No subheader,Krimi,Content not found,2024-08-12T13:03:27.261Z
...,...,...,...,...,...,...,...
842,https://nyheder.tv2.dk/udland/2018-05-20-trump...,tv-overvågn,Trump kræver undersøgelse af påstået FBI-overv...,"Han vil have undersøgt, om eventuel overvågnin...",Udland,"USA's præsident, Donald Trump, kræver, at det ...",2018-05-20T18:39:56.000Z
843,https://nyheder.tv2.dk/video/cmlnaXRna2FtZXJhY...,tv-overvågn,"Ups, vi kan ikke finde siden - men vi vil gern...",No subheader,No category,Content not found,No date
844,https://nyheder.tv2.dk/samfund/2017-10-04-smer...,tv-overvågn,Smertestillende medicin sættes under skarp ove...,Lægemiddelstyrelsen vil holde bedre øje med fo...,Samfund,Det stærke smertestillende medicinpræparat tra...,2017-10-04T05:57:58.000Z
845,https://nyheder.tv2.dk/politik/2017-08-17-amne...,tv-overvågn,Amnesty vil have strammere eksportregler for o...,No subheader,Politik,Dansk overvågningsteknologi må ikke havne i de...,2017-08-17T02:55:01.000Z


In [ ]:
# Remove rows when 'Content' column has 'Content not found'
df = df[df['Content'] != 'Content not found']
df

,URL,Keyword,Header,Subheader,Category,Content,Date
0,https://nyheder.tv2.dk/krimi/2024-08-08-rigspo...,facial recognition,Rigspolitichef forsøger at berolige befolkning...,Ansigtsgenkendelse vil spare politiet ressourc...,Krimi,Det vil gøre politiets arbejde betydeligt nemm...,2024-08-08T14:16:16.655Z
1,https://nyheder.tv2.dk/tech/2019-01-20-der-kan...,facial recognition,Der kan være en bagtanke med '10yearchallenge'...,Man kan bruge ansigtsgenkendelse til mere og m...,Tech,"Mærkelige frisurer, orange kulør fra solariet ...",2019-01-20T15:51:09.000Z
2,https://echo.tv2.dk/samfund/2020-01-07-24-aari...,facial recognition,24-årig kineser om omdiskuteret teknologi: - D...,"I Kina bruges ansigtsgenkendelse flittigt, men...",Samfund,Den 25-årige kinesiske studerende Hong Yen får...,2020-01-07T09:45:31.000Z
3,https://nyheder.tv2.dk/2015-02-02-forbrugerraa...,facial recognition,Forbrugerrådet Tænk til lufthavn: Begrund pass...,No subheader,Nyheder,Københavns Lufthavn har netop fået sat reklame...,2015-02-02T15:43:41.000Z
4,https://nyheder.tv2.dk/tech/2009-08-04-test-et...,facial recognition,Test: Et helt unikt feriekamera,No subheader,Tech,Jeg har ikke tal på hvor mange ferier og andre...,2009-08-04T12:50:00.000Z
...,...,...,...,...,...,...,...
445,https://nyheder.tv2.dk/samfund/2020-01-18-drab...,overvågningssamfund kamera,Drabssag får sygehus til at overvåge medicinrum,Videoovervågning på afdelingen er en lære af d...,Samfund,Som en konsekvens af sagen fra Nykøbing Falste...,2020-01-18T21:35:42.000Z
446,https://nyheder.tv2.dk/krimi/2019-12-25-par-va...,overvågningssamfund kamera,"Par var på vej til julemiddag, da kamera i stu...","Politiet reagerede hurtigt, men tog ikke tyven...",Krimi,En maskeret mand kommer ind ad døren. Han sluk...,2019-12-25T21:26:35.000Z
447,https://nyheder.tv2.dk/samfund/2019-12-25-tyve...,overvågningssamfund kamera,Tyve gjorde indbrudsforsøg hos minister juleaf...,Minister Pernille Rosenkrantz-Theil kom hjem f...,Samfund,Mens mere end 450 private hjem har haft indbru...,2019-12-25T21:34:37.000Z
448,https://nyheder.tv2.dk/lokalt/2019-12-18-danma...,overvågningssamfund kamera,Danmark indfører overvågning af fiskeri - fisk...,Danmark indfører som det første land i Europa ...,Nyheder,Danmark indfører elektronisk monitorering af f...,2019-12-18T07:09:03.000Z


In [ ]:
df['merged_content']= df['Header'] + ' ' + df['Subheader'] + ' ' + df['Content']

In [ ]:
# fixing date
df_filtered['Date'] = df_filtered['date'].str[:10]
df_filtered.columns = df_filtered.columns.str.lower()
df_filtered

<ipython-input-10-2f686ad69943>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Date'] = df_filtered['date'].str[:10]


,url,keyword,header,subheader,category,content,date,date
0,https://nyheder.tv2.dk/krimi/2024-08-08-rigspo...,facial recognition,Rigspolitichef forsøger at berolige befolkning...,Ansigtsgenkendelse vil spare politiet ressourc...,Krimi,Det vil gøre politiets arbejde betydeligt nemm...,2024-08-08,2024-08-08
1,https://nyheder.tv2.dk/tech/2019-01-20-der-kan...,facial recognition,Der kan være en bagtanke med '10yearchallenge'...,Man kan bruge ansigtsgenkendelse til mere og m...,Tech,"Mærkelige frisurer, orange kulør fra solariet ...",2019-01-20,2019-01-20
2,https://echo.tv2.dk/samfund/2020-01-07-24-aari...,facial recognition,24-årig kineser om omdiskuteret teknologi: - D...,"I Kina bruges ansigtsgenkendelse flittigt, men...",Samfund,Den 25-årige kinesiske studerende Hong Yen får...,2020-01-07,2020-01-07
3,https://nyheder.tv2.dk/2015-02-02-forbrugerraa...,facial recognition,Forbrugerrådet Tænk til lufthavn: Begrund pass...,No subheader,Nyheder,Københavns Lufthavn har netop fået sat reklame...,2015-02-02,2015-02-02
4,https://nyheder.tv2.dk/tech/2009-08-04-test-et...,facial recognition,Test: Et helt unikt feriekamera,No subheader,Tech,Jeg har ikke tal på hvor mange ferier og andre...,2009-08-04,2009-08-04
...,...,...,...,...,...,...,...,...
445,https://nyheder.tv2.dk/samfund/2020-01-18-drab...,overvågningssamfund kamera,Drabssag får sygehus til at overvåge medicinrum,Videoovervågning på afdelingen er en lære af d...,Samfund,Som en konsekvens af sagen fra Nykøbing Falste...,2020-01-18,2020-01-18
446,https://nyheder.tv2.dk/krimi/2019-12-25-par-va...,overvågningssamfund kamera,"Par var på vej til julemiddag, da kamera i stu...","Politiet reagerede hurtigt, men tog ikke tyven...",Krimi,En maskeret mand kommer ind ad døren. Han sluk...,2019-12-25,2019-12-25
447,https://nyheder.tv2.dk/samfund/2019-12-25-tyve...,overvågningssamfund kamera,Tyve gjorde indbrudsforsøg hos minister juleaf...,Minister Pernille Rosenkrantz-Theil kom hjem f...,Samfund,Mens mere end 450 private hjem har haft indbru...,2019-12-25,2019-12-25
448,https://nyheder.tv2.dk/lokalt/2019-12-18-danma...,overvågningssamfund kamera,Danmark indfører overvågning af fiskeri - fisk...,Danmark indfører som det første land i Europa ...,Nyheder,Danmark indfører elektronisk monitorering af f...,2019-12-18,2019-12-18


In [ ]:
filtered_df = df[~df['url'].str.startswith('https://nyheder.tv2.dk/video/')]
filtered_df


,url,keyword,header,subheader,category,content,date,merged_content
0,https://nyheder.tv2.dk/krimi/2024-08-08-rigspo...,facial recognition,Rigspolitichef forsøger at berolige befolkning...,Ansigtsgenkendelse vil spare politiet ressourc...,Krimi,Det vil gøre politiets arbejde betydeligt nemm...,2024-08-08,Rigspolitichef forsøger at berolige befolkning...
1,https://nyheder.tv2.dk/tech/2019-01-20-der-kan...,facial recognition,Der kan være en bagtanke med '10yearchallenge'...,Man kan bruge ansigtsgenkendelse til mere og m...,Tech,"Mærkelige frisurer, orange kulør fra solariet ...",2019-01-20,FC København vil benytte sig af ansigtsgenkend...
2,https://echo.tv2.dk/samfund/2020-01-07-24-aari...,facial recognition,24-årig kineser om omdiskuteret teknologi: - D...,"I Kina bruges ansigtsgenkendelse flittigt, men...",Samfund,Den 25-årige kinesiske studerende Hong Yen får...,2020-01-07,Politiet tager hul på første projekt med ansig...
3,https://nyheder.tv2.dk/2015-02-02-forbrugerraa...,facial recognition,Forbrugerrådet Tænk til lufthavn: Begrund pass...,No subheader,Nyheder,Københavns Lufthavn har netop fået sat reklame...,2015-02-02,Lufthavn i Singapore erstatter pas med ansigts...
4,https://nyheder.tv2.dk/tech/2009-08-04-test-et...,facial recognition,Test: Et helt unikt feriekamera,No subheader,Tech,Jeg har ikke tal på hvor mange ferier og andre...,2009-08-04,"Ansigtsgenkendelse er på alle måder en ""god id..."
...,...,...,...,...,...,...,...,...
841,https://nyheder.tv2.dk/udland/2018-11-26-schwe...,tv-overvågn,Schweiz siger ja til velfærdsdetektiver og øge...,"64,7 procent af vælgere i Schweiz bakker op om...",Udland,Schweiziske vælgere stemmer ja til velfærdsdet...,2018-11-26,Schweiz siger ja til velfærdsdetektiver og øge...
842,https://nyheder.tv2.dk/udland/2018-05-20-trump...,tv-overvågn,Trump kræver undersøgelse af påstået FBI-overv...,"Han vil have undersøgt, om eventuel overvågnin...",Udland,"USA's præsident, Donald Trump, kræver, at det ...",2018-05-20,Trump kræver undersøgelse af påstået FBI-overv...
844,https://nyheder.tv2.dk/samfund/2017-10-04-smer...,tv-overvågn,Smertestillende medicin sættes under skarp ove...,Lægemiddelstyrelsen vil holde bedre øje med fo...,Samfund,Det stærke smertestillende medicinpræparat tra...,2017-10-04,Smertestillende medicin sættes under skarp ove...
845,https://nyheder.tv2.dk/politik/2017-08-17-amne...,tv-overvågn,Amnesty vil have strammere eksportregler for o...,No subheader,Politik,Dansk overvågningsteknologi må ikke havne i de...,2017-08-17,Amnesty vil have strammere eksportregler for o...


In [ ]:
filtered_df = filtered_df[~filtered_df['url'].str.startswith('https://sport.tv2.dk')]
filtered_df

,url,keyword,header,subheader,category,content,date,merged_content
0,https://nyheder.tv2.dk/krimi/2024-08-08-rigspo...,facial recognition,Rigspolitichef forsøger at berolige befolkning...,Ansigtsgenkendelse vil spare politiet ressourc...,Krimi,Det vil gøre politiets arbejde betydeligt nemm...,2024-08-08,Rigspolitichef forsøger at berolige befolkning...
1,https://nyheder.tv2.dk/tech/2019-01-20-der-kan...,facial recognition,Der kan være en bagtanke med '10yearchallenge'...,Man kan bruge ansigtsgenkendelse til mere og m...,Tech,"Mærkelige frisurer, orange kulør fra solariet ...",2019-01-20,FC København vil benytte sig af ansigtsgenkend...
2,https://echo.tv2.dk/samfund/2020-01-07-24-aari...,facial recognition,24-årig kineser om omdiskuteret teknologi: - D...,"I Kina bruges ansigtsgenkendelse flittigt, men...",Samfund,Den 25-årige kinesiske studerende Hong Yen får...,2020-01-07,Politiet tager hul på første projekt med ansig...
3,https://nyheder.tv2.dk/2015-02-02-forbrugerraa...,facial recognition,Forbrugerrådet Tænk til lufthavn: Begrund pass...,No subheader,Nyheder,Københavns Lufthavn har netop fået sat reklame...,2015-02-02,Lufthavn i Singapore erstatter pas med ansigts...
4,https://nyheder.tv2.dk/tech/2009-08-04-test-et...,facial recognition,Test: Et helt unikt feriekamera,No subheader,Tech,Jeg har ikke tal på hvor mange ferier og andre...,2009-08-04,"Ansigtsgenkendelse er på alle måder en ""god id..."
...,...,...,...,...,...,...,...,...
841,https://nyheder.tv2.dk/udland/2018-11-26-schwe...,tv-overvågn,Schweiz siger ja til velfærdsdetektiver og øge...,"64,7 procent af vælgere i Schweiz bakker op om...",Udland,Schweiziske vælgere stemmer ja til velfærdsdet...,2018-11-26,Schweiz siger ja til velfærdsdetektiver og øge...
842,https://nyheder.tv2.dk/udland/2018-05-20-trump...,tv-overvågn,Trump kræver undersøgelse af påstået FBI-overv...,"Han vil have undersøgt, om eventuel overvågnin...",Udland,"USA's præsident, Donald Trump, kræver, at det ...",2018-05-20,Trump kræver undersøgelse af påstået FBI-overv...
844,https://nyheder.tv2.dk/samfund/2017-10-04-smer...,tv-overvågn,Smertestillende medicin sættes under skarp ove...,Lægemiddelstyrelsen vil holde bedre øje med fo...,Samfund,Det stærke smertestillende medicinpræparat tra...,2017-10-04,Smertestillende medicin sættes under skarp ove...
845,https://nyheder.tv2.dk/politik/2017-08-17-amne...,tv-overvågn,Amnesty vil have strammere eksportregler for o...,No subheader,Politik,Dansk overvågningsteknologi må ikke havne i de...,2017-08-17,Amnesty vil have strammere eksportregler for o...
